In [35]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/cloudera/parcels/Anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_232-cloudera/jre"
os.environ["SPARK_HOME"]="/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2-1.cdh5.13.3.p0.316101/lib/spark2/"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [36]:
from pyspark.sql import SparkSession

In [37]:
spark = SparkSession.builder.appName('etl').master("local").enableHiveSupport().getOrCreate()
spark

In [51]:
#Specifying the schema instead of inferring it 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType

fileSchema = StructType([StructField('year', IntegerType(),True),
                        StructField('month', StringType(),True),
                        StructField('day', IntegerType(),True),
                        StructField('weekday', StringType(),True),
                        StructField('hour', IntegerType(),True),
                        StructField('atm_status', StringType(),True),
                        StructField('atm_id', IntegerType(),True),
                        StructField('atm_manufacturer', StringType(),True),
                        StructField('atm_location', StringType(),True),
                        StructField('atm_streetname', StringType(),True),
                        StructField('atm_street_number', IntegerType(),True),
                        StructField('atm_zipcode', IntegerType(),True),
                        StructField('atm_lat', DoubleType(),True),
                        StructField('atm_lon', DoubleType(),True),
                        StructField('currency', StringType(),True),
                        StructField('card_type', StringType(),True),
                        StructField('transaction_amount', IntegerType(),True),
                        StructField('service', StringType(),True),
                        StructField('message_code', StringType(),True),
                        StructField('message_text', StringType(),True),
                        StructField('weather_lat', DoubleType(),True),
                        StructField('weather_lon', DoubleType(),True),
                        StructField('weather_city_id', IntegerType(),True),
                        StructField('weather_city_name', StringType(),True),
                        StructField('temp', DoubleType(),True),
                        StructField('pressure', IntegerType(),True),
                        StructField('humidity', IntegerType(),True),
                        StructField('wind_speed', IntegerType(),True),
                        StructField('wind_deg', IntegerType(),True),
                        StructField('rain_3h', DoubleType(),True),
                        StructField('clouds_all', IntegerType(),True),
                        StructField('weather_id', IntegerType(),True),
                        StructField('weather_main', StringType(),True),
                        StructField('weather_description', StringType(),True)
                        ])



In [52]:
df2 = spark.read.load("etl/part-m-00000", format="csv", sep=",", schema = fileSchema, header="false")

In [55]:
# Check count of records in the dataframe
df2.count()

2468572

In [53]:
df2.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: s

In [54]:
df2.show()

+----+-------+---+-------+----+----------+------+----------------+------------------+--------------------+-----------------+-----------+-------+-------+--------+--------------------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|      atm_location|      atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|           card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------------+--------------------+-----------------+-----------+-------+-------+--------+--------------------+------

In [56]:
# Create DIM_LOCATION DF
dim_location = df2[df2['atm_location'],df2['atm_streetname'],df2['atm_street_number'],df2['atm_zipcode'],df2['atm_lat'],df2['atm_lon']]
dim_location = dim_location.withColumnRenamed('atm_location', 'location')\
.withColumnRenamed('atm_streetname', 'streetname').withColumnRenamed('atm_street_number', 'street_number')\
.withColumnRenamed('atm_zipcode', 'zipcode').withColumnRenamed('atm_lat', 'lat').withColumnRenamed('atm_lon', 'lon')

In [57]:
# Drop duplicates from DIM_LOCATION
from pyspark.sql.functions import monotonically_increasing_id
dim_location = dim_location.dropDuplicates(['location', 'streetname', 'street_number', 'zipcode'])
dim_location = dim_location.withColumn('location_id', monotonically_increasing_id())

In [25]:
dim_location.head()

Row(location='Horsens', streetname='GrÃƒÂ¸nlandsvej', street_number=5, zipcode=8700, lat=55.859, lon=9.854, location_id=0)

In [59]:
# Check DIM_LOCATION count of records
dim_location.count()

109

In [60]:
# Create DIM_CARD_TYPE DF
dim_card_type = df2[['card_type']]
dim_card_type = dim_card_type.dropDuplicates()

In [61]:
# Add a primary key for card_type
from pyspark.sql.functions import monotonically_increasing_id
dim_card_type = dim_card_type.withColumn('card_type_id', monotonically_increasing_id())

In [27]:
dim_card_type.head()

Row(card_type='Dankort - on-us', card_type_id=17179869184)

In [63]:
# check count of DIM_CARD_TYPE
dim_card_type.count()

12

In [64]:
# Create DIM_ATM DF
atm_temp = df2[df2['atm_id'],df2['atm_manufacturer']]
cond = [df2.atm_lat == dim_location.lat, df2.atm_lon == dim_location.lon]
atm_temp = df2.join(dim_location, cond)
dim_atm = atm_temp[atm_temp['atm_id'],atm_temp['atm_manufacturer'], atm_temp['location_id']]
dim_atm = dim_atm.withColumn('atm_number', dim_atm['atm_id'])
dim_atm = dim_atm.withColumnRenamed('location_id', 'atm_location_id')

In [65]:
# Drop duplicate data
dim_atm = dim_atm.dropDuplicates()

In [66]:
# Check teh count of dim_atm
dim_atm.count()

156

In [67]:
dim_atm.head()

Row(atm_id=70, atm_manufacturer='Diebold Nixdorf', atm_location_id=163208757248, atm_number=70)

In [68]:
# Create DIM_DATE DF
from pyspark.sql.functions import *
dim_date = df2[df2['year'], df2['month'], df2['day'], df2['hour'], df2['weekday']]
dim_date = dim_date.dropDuplicates()
dim_date = dim_date.withColumn('full_date_time',to_date(concat_ws('-', df2.year, df2.month,df2.day)))
dim_date = dim_date.withColumn('date_id', monotonically_increasing_id())

In [69]:

dim_date.count()

8685

In [70]:
dim_date.head()

Row(year=2017, month='January', day=1, hour=9, weekday='Sunday', full_date_time=None, date_id=0)

In [71]:
# Create FACT_ATM_TRANS
fact_atm_trans = df2
fact_atm_trans = fact_atm_trans.withColumn('trans_id', monotonically_increasing_id())


In [72]:
# Join with DIM_LOCATION
dim_cond = [df2.atm_lat == dim_location.lat, df2.atm_lon == dim_location.lon]
fact_atm_trans = fact_atm_trans.join(dim_location, dim_cond, 'left')


In [73]:
# Rename location_id to weather_loc_id
fact_atm_trans = fact_atm_trans.withColumnRenamed('location_id', 'weather_loc_id')

In [74]:
fact_atm_trans = fact_atm_trans.dropDuplicates(['trans_id'])

In [75]:
# Join with DIM_ATM
fact_atm_trans = fact_atm_trans.join(dim_atm[['atm_number', 'atm_manufacturer', 'atm_location_id']], fact_atm_trans.atm_id == dim_atm.atm_number, 'left')

In [76]:
fact_atm_trans = fact_atm_trans.dropDuplicates(['trans_id'])

In [77]:
# Join with DIM_DATE
date_cond = [fact_atm_trans.year == dim_date.year, fact_atm_trans.month == dim_date.month, \
            fact_atm_trans.day == dim_date.day, fact_atm_trans.hour == dim_date.hour, \
            fact_atm_trans.weekday == dim_date.weekday]
fact_atm_trans = fact_atm_trans.join(dim_date, date_cond, 'left')

In [78]:
fact_atm_trans = fact_atm_trans.dropDuplicates(['trans_id'])

In [79]:
# Join with DIM_CARD_TYPE
fact_atm_trans = fact_atm_trans.join(dim_card_type, fact_atm_trans.card_type == dim_card_type.card_type, 'left')

In [80]:
fact_atm_trans = fact_atm_trans.dropDuplicates(['trans_id'])


In [81]:
# Remove unnecessary columns
fact_atm_trans_temp = fact_atm_trans[['trans_id','atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 'atm_status', \
                                'currency', 'service', 'transaction_amount', 'message_code', 'message_text', \
                                'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']]

In [82]:
fact_atm_trans_temp.show(5)

+--------+------+--------------+-------------+-------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|      date_id| card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------------+-------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|      26|    20|  841813590016|1649267441664|1056561954816|    Active|     DKK|Withdrawal|              7573|        null|        null|   1.26|        92|       500|        Rain|         light rain|
|      29|    21|  309237645312|1649267441664| 326417514496|    Active|     DKK|Withdrawal|              4607|        null|        null|   0.59|        92|       500|        Rain|         light rain|


In [34]:
fact_atm_trans_temp.head()

Row(trans_id=26, atm_id=None, weather_loc_id=None, date_id=None, card_type_id=None, atm_status=None, currency=None, service=None, transaction_amount=None, message_code=None, message_text=None, rain_3h=None, clouds_all=None, weather_id=None, weather_main=None, weather_description=None)

In [83]:
# Loading DIM_LOCATION table to S3
dim_location.write.csv("s3a://upgrad7474redshiftbucket/assignment/dim_location", sep="|")

In [84]:
# Loading DIM_CARD_TYPE table to s3
dim_card_type.write.csv("s3a://upgrad7474redshiftbucket/assignment/dim_card_type", sep="|")

In [85]:
# Loading DIM_ATM table to s3
dim_atm.write.csv("s3a://upgrad7474redshiftbucket/assignment/dim_atm", sep="|")

In [86]:
# Loading DIM_DATE table to s3
dim_date.write.csv("s3a://upgrad7474redshiftbucket/assignment/dim_date", sep="|")

In [87]:
# Loading FACT_ATM_TRANS table to s3
fact_atm_trans_temp.write.csv("s3a://upgrad7474redshiftbucket/assignment/fact_atm_trans", sep="|")